## Event camera demo
**By Cedric Scheerlinck**  
Tested on python3.7.5  
Description: load and visualize events from a small dataset.  
Based on https://github.com/cedric-scheerlinck/dvs_image_reconstruction


Note: If interact doesn't work, try enabling ipywidgets and restarting jupyter notebook:  
```jupyter nbextension enable --py widgetsnbextension```

In [1]:
import numpy as np
from IPython.display import HTML
import time
import pandas as pd
import math
import ipywidgets as widgets
from ipywidgets import interact, fixed, interact_manual
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from matplotlib import rc
rc('animation', html='jshtml')

# local modules
from util import Timer, Event, normalize_image, animate, load_events

In [2]:
def high_pass_filter(events, cutoff_frequency=5):
    print('Reconstructing, please wait...')
    events_per_frame = 2e4
    with Timer('Reconstruction'):
        time_surface = np.zeros((height, width), dtype=np.float32)
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            beta = math.exp(-cutoff_frequency * (e.t - time_surface[e.y, e.x]))
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            time_surface[e.y, e.x] = e.t
            if i % events_per_frame == 0:
                beta = np.exp(-cutoff_frequency * (e.t - time_surface))
                image_state *= beta
                time_surface.fill(e.t)
                image_list.append(np.copy(image_state))
    return animate(image_list)

def high_pass_filter_simple(events, beta=1.0):
    print('Reconstructing, please wait...')
    events_per_frame = 2e4
    with Timer('Reconstruction (simple)'):
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            if i % events_per_frame == 0:
                image_list.append(np.copy(image_state))
    return animate(image_list)


In [3]:
with Timer('Loading'):
    n_events = 5e5
    path_to_events = 'data/boxes_6dof/events.zip'
    event_list, width, height = load_events(path_to_events, n_events)        

width, height: 240, 180
Loaded 0.50M events
Loading: 0.95s


## Image reconstruction
Run the cell, then click ```Run Interact``` and wait for the animation to appear to see the result.  
Use the slider then click ```Run Interact``` again to try different settings.

### High pass filter (simple)
Equation: $L^{k+1}(x, y) = \beta L^k(x, y) + p,\ \beta \in [0, 1],\ p \in \{-1, 1\}$, where $L$ is the log image intensity and $p$ is the event polarity.

#### Set ```beta = 1``` for direct integration. Try lowering beta to see what happens.

In [4]:
interact_manual(high_pass_filter_simple, events=fixed(event_list), beta=(0, 1, 0.01));

interactive(children=(FloatSlider(value=1.0, description='beta', max=1.0, step=0.01), Button(description='Run …

### High pass filter
Below is the high pass filter as described in: https://cedric-scheerlinck.github.io/files/2018_scheerlinck_continuous-time_intensity_estimation.pdf  
Equation: $L^{k+1}(x, y) = \exp(-\alpha \Delta t) L^k(x, y) + p$, where $\alpha$ is the cutoff frequency and $\Delta t$ is the time since the last event at the same pixel.

In [5]:
interact_manual(high_pass_filter, events=fixed(event_list), cutoff_frequency=(0, 10, 0.01));

interactive(children=(FloatSlider(value=5.0, description='cutoff_frequency', max=10.0, step=0.01), Button(desc…